In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter

# Set style for professional plots
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'sans-serif']
plt.rcParams['font.size'] = 12

def create_antiferromagnetic_pattern(size=8):
    """Create a checkerboard pattern representing antiferromagnetic Ising spins."""
    pattern = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            # Checkerboard pattern: +1 for even sum of indices, -1 for odd
            pattern[i, j] = 1 if (i + j) % 2 == 0 else -1
    return pattern

def create_energy_landscape(pattern, resolution_factor=4):
    """Create 3D energy landscape with ridges at boundaries and valleys at centers."""
    
    size = pattern.shape[0]
    # Create higher resolution grid for smooth surface
    high_res_size = size * resolution_factor
    
    # Create coordinate grids
    x = np.linspace(0, size-1, high_res_size)
    y = np.linspace(0, size-1, high_res_size)
    X, Y = np.meshgrid(x, y)
    
    # Initialize height map
    Z = np.zeros_like(X)
    
    # For each high-res point, calculate energy based on distance to boundaries
    for i in range(high_res_size):
        for j in range(high_res_size):
            # Current position in original grid coordinates
            orig_i = Y[i, j]
            orig_j = X[i, j]
            
            # Find which cell we're in
            cell_i = int(orig_i)
            cell_j = int(orig_j)
            
            # Handle boundary conditions
            cell_i = min(cell_i, size-1)
            cell_j = min(cell_j, size-1)
            
            # Position within the cell (0 to 1)
            local_i = orig_i - cell_i
            local_j = orig_j - cell_j
            
            # Distance to center of cell (0 at center, ~0.7 at corners)
            dist_to_center = np.sqrt((local_i - 0.5)**2 + (local_j - 0.5)**2)
            
            # Energy increases near boundaries (edges of cells)
            # Valley at center (dist_to_center = 0), ridge at edges
            base_energy = dist_to_center * 2  # Scale factor for height
            
            # Add extra energy at actual spin boundaries
            spin_current = pattern[cell_i, cell_j]
            
            # Check neighboring cells for energy penalties
            boundary_penalty = 0
            neighbors = [
                (cell_i-1, cell_j), (cell_i+1, cell_j),
                (cell_i, cell_j-1), (cell_i, cell_j+1)
            ]
            
            for ni, nj in neighbors:
                if 0 <= ni < size and 0 <= nj < size:
                    if pattern[ni, nj] != spin_current:
                        # Distance to this boundary
                        if ni != cell_i:  # Vertical boundary
                            dist_to_boundary = abs(local_i - (0 if ni < cell_i else 1))
                        else:  # Horizontal boundary
                            dist_to_boundary = abs(local_j - (0 if nj < cell_j else 1))
                        
                        # Add energy penalty near boundaries
                        boundary_penalty += np.exp(-dist_to_boundary * 8) * 1.5
            
            Z[i, j] = base_energy + boundary_penalty
    
    # Smooth the surface slightly
    Z = gaussian_filter(Z, sigma=0.5)
    
    return X, Y, Z

def create_color_map(pattern, X, Y):
    """Create color map for the 3D surface based on spin values."""
    
    size = pattern.shape[0]
    colors = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Find which original cell this point belongs to
            cell_i = int(Y[i, j])
            cell_j = int(X[i, j])
            
            # Handle boundary conditions
            cell_i = min(cell_i, size-1)
            cell_j = min(cell_j, size-1)
            
            colors[i, j] = pattern[cell_i, cell_j]
    
    return colors

def plot_3d_ising_landscape(pattern, title="3D Ising Model Energy Landscape", 
                           save_name="ising_3d_landscape", elevation=30, azimuth=45):
    """Create 3D visualization of Ising model energy landscape."""
    
    # Create the energy landscape
    X, Y, Z = create_energy_landscape(pattern, resolution_factor=4)
    colors = create_color_map(pattern, X, Y)
    
    # Create 3D plot
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create custom colormap: red for spin down (-1), green for spin up (+1)
    cmap_colors = ['#FF6B6B', '#6BFF6B']  # Red, Green
    cmap = ListedColormap(cmap_colors)
    
    # Plot the surface
    surf = ax.plot_surface(X, Y, Z, facecolors=cmap(colors/2 + 0.5), 
                          alpha=0.9, linewidth=0.5,
                          antialiased=True, shade=True)
    
    # Customize the plot
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('X Position', fontsize=12, fontweight='bold')
    ax.set_ylabel('Y Position', fontsize=12, fontweight='bold')
    ax.set_zlabel('Energy', fontsize=12, fontweight='bold')
    
    # Set viewing angle
    ax.view_init(elev=elevation, azim=azimuth)
    
    # Add custom colorbar
    m = plt.cm.ScalarMappable(cmap=cmap)
    m.set_array([-1, 1])
    cbar = plt.colorbar(m, ax=ax, shrink=0.8, aspect=20)
    cbar.set_ticks([-1, 1])
    cbar.set_ticklabels(['Spin ↓ (Red)', 'Spin ↑ (Green)'])
    
    # Improve aesthetics
    ax.grid(True, alpha=0.3)
    
    # Save the plot
    plt.savefig(f'{save_name}.png', dpi=600, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    print(f"✓ Saved 3D plot as '{save_name}.png'")
    
    plt.tight_layout()
    return fig, ax

def plot_multiple_views(pattern, save_name="ising_3d_multiple_views"):
    """Create multiple viewing angles of the 3D energy landscape."""
    
    fig = plt.figure(figsize=(20, 5))
    
    # Different viewing angles
    views = [
        (30, 45, "Standard View"),
        (60, 30, "High Angle"),
        (15, 135, "Side View"),
        (45, 90, "Profile View")
    ]
    
    X, Y, Z = create_energy_landscape(pattern, resolution_factor=3)
    colors = create_color_map(pattern, X, Y)
    
    # Custom colormap
    cmap_colors = ['#FF6B6B', '#6BFF6B']  # Red, Green
    cmap = ListedColormap(cmap_colors)
    
    for i, (elev, azim, view_name) in enumerate(views):
        ax = fig.add_subplot(1, 4, i+1, projection='3d')
        
        # Plot surface
        surf = ax.plot_surface(X, Y, Z, facecolors=cmap(colors/2 + 0.5),
                              alpha=0.8, linewidth=0.3,
                              antialiased=True, shade=True)
        
        # Customize
        ax.set_title(view_name, fontsize=12, fontweight='bold')
        ax.view_init(elev=elev, azim=azim)
        ax.grid(True, alpha=0.3)
        
        # Remove axis labels for cleaner look in multi-view
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
    
    plt.suptitle('3D Ising Energy Landscape - Multiple Views', 
                 fontsize=16, fontweight='bold')
    
    # Save the plot
    plt.savefig(f'{save_name}.png', dpi=600, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    print(f"✓ Saved multi-view plot as '{save_name}.png'")
    
    plt.tight_layout()
    return fig

def create_interactive_3d_demo():
    """Create an interactive demo showing the energy landscape concept."""
    
    print("3D ISING MODEL ENERGY LANDSCAPE DEMO")
    print("="*45)
    print("Creating 3D visualization of energy landscape...")
    print("• Valleys = Low energy (stable spin regions)")
    print("• Ridges = High energy (domain boundaries)")
    print()
    
    # Create antiferromagnetic pattern
    pattern = create_antiferromagnetic_pattern(8)
    
    # Create main 3D plot
    print("1. Creating main 3D energy landscape...")
    fig1, ax1 = plot_3d_ising_landscape(
        pattern, 
        title="Antiferromagnetic Ising Model - 3D Energy Landscape",
        save_name="ising_3d_main"
    )
    plt.show()
    
    # Create multiple views
    print("\n2. Creating multiple viewing angles...")
    fig2 = plot_multiple_views(pattern, save_name="ising_3d_views")
    plt.show()
    
    print("\n" + "="*45)
    print("3D VISUALIZATION INTERPRETATION:")
    print("✓ Red regions: Spin ↓ (-1) domains")
    print("✓ Green regions: Spin ↑ (+1) domains") 
    print("✓ Valleys: Stable energy minima within domains")
    print("✓ Ridges: High energy at domain boundaries")
    print("✓ Perfect for showing why checkerboard is ground state!")
    
    return pattern

def compare_2d_vs_3d():
    """Create side-by-side comparison of 2D and 3D visualizations."""
    
    pattern = create_antiferromagnetic_pattern(6)
    
    fig = plt.figure(figsize=(20, 8))
    
    # 2D heatmap
    ax1 = fig.add_subplot(121)
    
    # Custom colormap
    colors = ['#FF4444', '#4444FF']
    cmap_2d = ListedColormap(colors)
    
    im = ax1.imshow(pattern, cmap=cmap_2d, vmin=-1, vmax=1, 
                    interpolation='nearest', aspect='equal')
    
    # Add grid lines
    for i in range(pattern.shape[0] + 1):
        ax1.axhline(i - 0.5, color='black', linewidth=2)
    for j in range(pattern.shape[1] + 1):
        ax1.axvline(j - 0.5, color='black', linewidth=2)
    
    ax1.set_title('2D Checkerboard Pattern', fontsize=14, fontweight='bold')
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    # 3D surface
    ax2 = fig.add_subplot(122, projection='3d')
    
    X, Y, Z = create_energy_landscape(pattern, resolution_factor=3)
    colors_3d = create_color_map(pattern, X, Y)
    
    cmap_3d = ListedColormap(['#FF6B6B', '#6BFF6B'])  # Red, Green
    
    surf = ax2.plot_surface(X, Y, Z, facecolors=cmap_3d(colors_3d/2 + 0.5),
                           alpha=0.9, linewidth=0.3,
                           antialiased=True, shade=True)
    
    ax2.set_title('3D Energy Landscape', fontsize=14, fontweight='bold')
    ax2.view_init(elev=35, azim=45)
    
    plt.suptitle('Ising Model: 2D Pattern vs 3D Energy Landscape', 
                 fontsize=16, fontweight='bold')
    
    # Save comparison
    plt.savefig('ising_2d_vs_3d_comparison.png', dpi=600, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    print("✓ Saved comparison as 'ising_2d_vs_3d_comparison.png'")
    
    plt.tight_layout()
    plt.show()
    
    return fig

def main():
    """Run the complete 3D Ising visualization demo."""
    
    # Run interactive demo
    pattern = create_interactive_3d_demo()
    
    print("\n3. Creating 2D vs 3D comparison...")
    compare_2d_vs_3d()
    
    print("\nQUICK 3D LANDSCAPE CREATION:")
    print("For presentations, use:")
    print("pattern = create_antiferromagnetic_pattern(8)")
    print("fig, ax = plot_3d_ising_landscape(pattern)")
    print()
    print("Perfect for explaining:")
    print("🎯 Why checkerboard is ground state")
    print("⚡ Energy costs of domain boundaries")
    print("🏔️ Energy landscape topology")
    print("🔬 Phase transition mechanisms")

if __name__ == "__main__":
    main()